In [2]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import numpy as np
import pickle

# Initialize the Flask app
app = Flask(__name__)

# Load the model
xgb_model = pickle.load(open('medicare.pkl', 'rb'))

# Load the data for helper function
data = pd.read_csv('medicare.csv')

# Encode target labels
le = pickle.load(open('medicare.pkl', 'rb'))

# Helper function for disease details
def helper(dis):
    disease_data = data[data['Disease'] == dis]
    desc = disease_data['Description'].values[0] if not disease_data['Description'].isna().all() else "No description available"
    pre = [disease_data['Precaution_1'].values[0], disease_data['Precaution_2'].values[0], disease_data['Precaution_3'].values[0], disease_data['Precaution_4'].values[0]]
    pre = [p for p in pre if pd.notna(p)]
    med = [disease_data['Medication'].values[0]] if not disease_data['Medication'].isna().all() else []
    die = [disease_data['Diet'].values[0]] if not disease_data['Diet'].isna().all() else []
    wrkout = [disease_data['workout'].values[0]] if not disease_data['workout'].isna().all() else []
    return desc, pre, med, die, wrkout

# Prediction function
def get_predicted_value(patient_symptoms):
    patient_symptoms = [s.lower() for s in patient_symptoms]
    input_vector = pd.DataFrame(columns=data.columns[:-1])
    input_vector.loc[0] = 0
    unknown_symptoms = []
    for symptom in patient_symptoms:
        if symptom in input_vector.columns:
            input_vector[symptom] = 1
        else:
            unknown_symptoms.append(symptom)
    if unknown_symptoms:
        return None, unknown_symptoms
    input_vector = input_vector.to_numpy()
    prediction = xgb_model.predict(input_vector)[0]
    decoded_prediction = le.inverse_transform([prediction])[0]
    return decoded_prediction, None

# Define the home route
@app.route('/')
def home():
    return render_template('index.html')

# Define the prediction route
@app.route('/predict', methods=['POST'])
def predict():
    symptoms = request.form.get('symptoms')
    user_symptoms = [s.strip() for s in symptoms.split(',')]
    predicted_disease, unknown_symptoms = get_predicted_value(user_symptoms)
    if predicted_disease:
        desc, pre, med, die, wrkout = helper(predicted_disease)
        return render_template('result.html', disease=predicted_disease, description=desc, precautions=pre, medications=med, diets=die, workouts=wrkout)
    else:
        return render_template('error.html', unknown_symptoms=unknown_symptoms)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\L E N O V O\jupyternotebook\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Personalized Medicine Recommender</title>
</head>
<body>
    <h1>Enter Symptoms</h1>
    <form action="/predict" method="post">
        <label for="symptoms">Symptoms (comma separated):</label>
        <input type="text" id="symptoms" name="symptoms">
        <button type="submit">Predict</button>
    </form>
</body>
</html>



SyntaxError: invalid syntax (125492969.py, line 1)